# Miary
---
obliczenie miar zagregowanych (X) dla całej sieci

korzysta z plików .csv obliczonych w pliku `obliczenia.ipynb`

In [1]:
import pandas as pd
import numpy as np

#### parametrty do scenariuszy

In [2]:
Bs= [60, 75, 90, 105, 120] 
t_ks =[30,40,50,60,70,80,90]
modes = ["PT","Car"]
aggs = ['mean','min','max','std','median', 'p_25', 'p_75']

In [5]:
def readfull(ch,B,t_k,mode, agg, filtr = 'filter', col = 'A_i_', folder = 'data//res//'):
    """ otworz odpowiedni plik csv i załaduj do pandas"""
    name = "{}_{}_{}_{}_{}_".format(ch,mode,str(B),str(t_k), filtr)
    path = folder+col+name+".csv"  
    df = pd.read_csv(path, index_col=0)
    return df
df = readfull('C',120,60,'PT',None)


In [6]:
df.head()

,A_i_C_PT_120_60_filter_mean,A_i_C_PT_120_60_filter_min,A_i_C_PT_120_60_filter_max,A_i_C_PT_120_60_filter_std,A_i_C_PT_120_60_filter_median,A_i_C_PT_120_60_filter_p_25,A_i_C_PT_120_60_filter_p_75
i,,,,,,,
1,69.17573,60.000736,108.21311,7.135563,67.585266,63.377483,73.35868
3,69.40398,60.002570,118.85825,7.618120,67.519100,63.384670,73.70188
4,69.45826,60.000840,119.21311,7.762724,67.611120,63.432260,73.56185
5,69.45277,60.001083,106.93256,7.417631,67.613020,63.523132,73.84390
6,69.81810,60.000850,118.93256,7.831326,67.993515,63.587320,74.28331


In [24]:
def readone(ch,B,t_k,mode, agg, filtr = 'filter', col = 'A_i_', folder = 'data//res//'):
    """ otworz odpowiedni plik csv i załaduj do pandas"""
    name = "{}_{}_{}_{}_{}_".format(ch,mode,str(B),str(t_k), filtr)
    path = folder+'A_i_'+name+".csv"  
    df = pd.read_csv(path, index_col=0)
    return df[col+name+agg]

def X(ch,B,t_k, filtr = 'filter', col = 'A_i_'):
    """ otworz dla pliki csv i oblicz miatę X dla odpowiednich parametrów"""
    a = readone(ch,B,t_k,modes[0],aggs[0], filtr, col)
    b = readone(ch,B,t_k,modes[1],aggs[0], filtr, col)
    i = a.index.union(b.index)
    a = a.reindex(i).fillna(0)
    b = b.reindex(i).fillna(0)
    df =  pd.DataFrame((a - b)/(a  + b ))
    df.columns=['mean']
    for agg in aggs[1:]:
        a = readone(ch,B,t_k,modes[0],agg)
        b = readone(ch,B,t_k,modes[1],agg)
        i = a.index.union(b.index)
        a = a.reindex(i).fillna(0)
        b = b.reindex(i).fillna(0)
        df[agg] = (a - b)/(a  + b )
    return df
    

#### pętla dla wszysctkich scenariuszy 
* załaduj pliki
* oblicz miarę
* zapisz plik

In [25]:
folder = 'data//res//'
for ch in ["C","H"]:
    for B in Bs:
        for t_k in t_ks:
            if B>t_k:
                for filtr in ['no_filter','filter']:
                    name = "{}_{}_{}_{}".format(ch,str(B),str(t_k), filtr)
                    path = folder+'X_'+name+".csv" 
                    X(ch,B,t_k,filtr).to_csv(path, sep = ";")         

IOError: File data//res//A_i_C_Car_120_30_no_filter_.csv does not exist

### 4.	Regional modal accessibility disparity, unweighted

In [9]:
res = pd.DataFrame(columns=['ch','B','t_k', 'agg', 'PT','Car','X', 'filter', 'weighted'])
for ch in ["C","H"]:
    for B in Bs:
            for t_k in t_ks:
                if B>t_k: 
                    for agg in aggs:
                        for filtr in ['filter', 'no_filter']:
                            for col in ["WA_i","A_i"]:
                                PT = readone(ch,B,t_k, 'PT', agg, filtr, col).mean()
                                Car =readone(ch,B,t_k, 'Car', agg, filtr, col).mean()
                                X = (PT - Car)/(PT + Car)
                                res.loc[res.shape[0]+1]=[ch,B,t_k, agg,PT, Car,X, filtr, col == "WA_i"]
                            
res.to_csv('data//res//zbiorcze.csv')

IOError: File data//res//WA_iC_PT_60_30_filter_.csv does not exist

In [9]:
res

,ch,B,t_k,agg,PT,Car,X,filter
1,C,60,30,mean,34.307557,33.834006,0.006950,filter
2,C,60,30,mean,34.680781,33.664040,0.014877,no_filter
3,C,60,30,min,30.283686,30.081581,0.003348,filter
4,C,60,30,min,30.478344,30.032646,0.007366,no_filter
5,C,60,30,max,48.153163,44.054482,0.044451,filter
6,C,60,30,max,48.926097,44.140344,0.051423,no_filter
7,C,60,30,std,3.888171,2.947398,0.137629,filter
8,C,60,30,std,4.137138,2.883291,0.178600,no_filter
9,C,60,30,median,33.217801,33.205235,0.000189,filter
10,C,60,30,median,33.469405,33.016526,0.006812,no_filter


----